In [4]:
#https://www.mygreatlearning.com/blog/real-time-face-detection/#sh1
#https://medium.com/@bipinadvani/face-recognition-and-blurring-in-webcam-using-cv2-python-5c4c589e6e59
#https://pythonprogramming.net/haar-cascade-face-eye-detection-python-opencv-tutorial/
#https://github.com/kurttepelikerim/FaceDetectionAnonymization/blob/master/FaceDetection.py
#https://docs.opencv.org/master/d4/d13/tutorial_py_filtering.html
#https://github.com/kunalgupta777/OpenCV-Face-Filters
#https://sublimerobots.com/2015/02/dancing-mustaches/
#https://github.com/charlielito/snapchat-filters-opencv/blob/master/README.md

# Face detection and anonymisation in real time video

In [1]:
import cv2
import numpy as np
import os

##  Load and configure Haar Cascade Classifiers

In [2]:
# location of OpenCV Haar Cascade Classifiers:
cascPath = os.path.dirname( cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
cascPath_eye = os.path.dirname( cv2.__file__) + "/data/haarcascade_eye.xml"
# build our cv2 Cascade Classifiers
faceCascade = cv2.CascadeClassifier(cascPath)
eye_cascade = cv2.CascadeClassifier(cascPath_eye)

## Load and configure images (.png with alpha transparency)

In [3]:
# Load our overlay image: mustache.png
mst = cv2.imread('moustache.png',-1)
filter_size_ratio_glasse = mst.shape[0] / mst.shape[1]
# Load our overlay image: eyes.png
gls = cv2.imread("eyes.png",-1)
filter_size_ratio_glasse = gls.shape[0] / gls.shape[1]
# Load our overlay image: black.png
blk = cv2.imread("black.png",-1)
filter_size_ratio_black = blk.shape[0] / blk.shape[1]

#Variable initialization
blur_faced = False
framed = False
mustache = False
framed_eyes = False
blur_faced_2 = False
blur_faced_3 = False
glasse = False
black = False

 # Main program loop

In [ ]:
# collect video input from first webcam on system
cap = cv2.VideoCapture(0)

while True:
    # Capture video feed
    ret,frame = cap.read()
    
    if(ret):
        
        # Create greyscale image from the video feed
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2BGRA)
        
        # Detect faces in input video stream
        f = faceCascade.detectMultiScale(gray, scaleFactor=1.1,minNeighbors=7)

        # Iterate over each face found
        for x,y,w,h in f:
            #region of interest of each frame
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            
           # Detect eyes in input video stream
            eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.35,minNeighbors=6)
            
            if blur_faced:
                # apply a gaussian blur on this new rectangle image
                roi_color = cv2.GaussianBlur(roi_color,(23, 23), 30)
                # merge this blurry rectangle to our final image
                frame[y:y+roi_color.shape[0], x:x+roi_color.shape[1]] = roi_color
                
            if blur_faced_2:
                # apply a median blur on this new rectangle image
                roi_color = cv2.medianBlur(roi_color,23)
                # merge this blurry rectangle to our final image
                frame[y:y+roi_color.shape[0], x:x+roi_color.shape[1]] = roi_color
                
            if blur_faced_3:
                # apply a Bileteral Filter on this new rectangle image
                roi_color = cv2.bilateralFilter(roi_color,23, 70, 70)
                # merge this blurry rectangle to our final image
                frame[y:y+roi_color.shape[0], x:x+roi_color.shape[1]] = roi_color 
                
            if framed:
                # Draw rectangle around the faces
                cv2.rectangle(frame,(x,y),(x+h,y+w),(0,255,0),2)
                
            if framed_eyes:
                #Iterate over each eyes found
                for (ex,ey,ew,eh) in eyes:
                    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                    
            if mustache:
                # w as face width h as face height
                mst_width = int(w*0.7)+1
                mst_height = int(h*0.2)+1
                
                # Resize the original image
                mst = cv2.resize(mst,(mst_width,mst_height))
        
                for i in range(int(0.7*h),int(0.7*h)+mst_height):
                    for j in range(int(0.2*w),int(0.2*w)+mst_width):
                        for k in range(3):
                            if mst[i-int(0.7*h)][j-int(0.2*w)][k] < 235:
                                frame[y+i][x+j][k] = mst[i-int(0.7*h)][j-int(0.2*w)][k]
            if glasse:
                #Iterate over each eyes found
                for (ex,ey,ew,eh) in eyes:
                    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                    roi_eyes = roi_gray[ey:ey + eh, ex:ex + ew]
                    # Resize the original image
                    gls = cv2.resize(gls.copy(),(ew, int(ew/filter_size_ratio_glasse)))

                    for i in range(gls.shape[0]):
                        for j in range(gls.shape[1]):
                            if gls[i,j][3] != 0:
                                roi_color[ey+i, ex+j] = gls[i,j] 
                                
            if black:
                #Iterate over each eyes found
                for (ex,ey,ew,eh) in eyes:
                    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                    roi_eyes = roi_gray[ey:ey + eh, ex:ex + ew]
                    # Resize the original image
                    blk = cv2.resize(blk.copy(),(ew, int(ew/filter_size_ratio_black)))

                    for i in range(blk.shape[0]):
                        for j in range(blk.shape[1]):
                            if blk[i,j][3] != 0:
                                roi_color[ey+i, ex+j] = blk[i,j]                    
                                
        frame = cv2.cvtColor(frame,cv2.COLOR_BGRA2BGR)  
        
        
        # Display the resulting frame    
        cv2.imshow('Face Detection - Projet INF573',frame)
    ch = 0xFF &cv2.waitKey(1)
    
    if ch == ord("c"):
        blur_faced = not blur_faced
        
    if ch == ord("f"):
        black = not black
        
    if ch == ord("g"):
        glasse = not glasse
        
    if ch == ord("e"):
        mustache = not mustache
        
    if ch == ord("d"):
        blur_faced_2 = not blur_faced_2
        
    if ch == ord("b"):
        framed_eyes = not framed_eyes

    if ch == ord("a"):
        framed = not framed
        
    if ch == ord("w"):
        blur_faced_3 = not blur_faced_3
        
    if ch==ord('s'):
        count=0
        count=count+1
        name="frame"+str(count)+".jpg"
        cv2.imwrite(name,frame)

    if ch ==ord("q"):
        break
        
        
# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()